# Projet électif python: Comparaisons générales sur les statistiques de NBA

In [1]:
# %pip install pandas numpy plotly requests time

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests
import plotly.express as px
import time

## Analyse des données des joueurs

### Récupération des données

On procède ici à un scrapping des données depuis le site officiel de la NBA.

On commence par un test avec l'url nécessaire pour récupérer ces données, ce qui nous permet de définir les en-têtes (`headers`) que l'on souhaite récupérer. Comme l'url ne eprmet de récupérer les données que d'une saison et d'un type de saison à la fois, on ajoute ces deux colonnes au `DataFrame` final afin de réaliser un fichier global contenant toutes les saisons et types de saisons souhaités.
Dans notre cas, on va récupérer les données de 2013 à 2024.

In [ ]:
test_url ='https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2022-23&SeasonType=Regular%20Season&StatCategory=PTS'

In [ ]:
r = requests.get(test_url).json()

In [ ]:
table_headers = r['resultSet']['headers']

In [ ]:
df_cols = ['Year', 'Season_type'] + table_headers

____

In [ ]:
headers ={
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6,ar;q=0.5,af;q=0.4',
    'Connection': 'keep-alive',
    'Host': 'stats.nba.com',
    'Origin': 'https://www.nba.com',
    'Referer': 'https://www.nba.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Mobile Safari/537.36',
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'sec-ch-ua-mobile': '?1',
    'sec-ch-ua-platform': '"Android"'
}

La méthode suivante permet de procéder au scrapping des données. Pour ce faire, on a défini plus haut un "headers", qui regroupe les informations supplémentaires envoyées avec la requête HTTP et fournissent des détails sur le client et ses préférences au serveur Web.
Ensuite, on crée un DataFrame vide, puis on parcourt différentes saisons et types de saison de la NBA. Pour chaque combinaison de saison et de type de saison, elle envoie une requête à l'API de la NBA pour obtenir les données. Les données récupérées sont ensuite ajoutées au DataFrame. Après avoir terminé le scrapping, les données sont enregistrées dans un fichier CSV.

In [ ]:
df = pd.DataFrame(columns=df_cols)
season_types = ['Regular%20Season', 'Playoffs']
years = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
begin = time.time()

for y in years :
    for s in season_types :
        api_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season='+y+'&SeasonType='+s+'&StatCategory=PTS'
        r= requests.get(url=api_url, headers=headers).json()
        temp_df1 = pd.DataFrame(r['resultSet']['rowSet'], columns=table_headers)
        temp_df2 = pd.DataFrame({'Year':[y for i in range(len(temp_df1))],
                                'Season_type':[s for i in range(len(temp_df1))]})
        temp_df3 = pd.concat([temp_df2, temp_df1], axis=1)
        df = pd.concat([df, temp_df3], axis=0)
        print(f'Scraping terminé pour la saison {y} et le type de saison {s}.')
        lag = np.random.uniform(low=5, high=40)
        print(f'Pause de {round(lag,1)} secondes avant de continuer...')
        time.sleep(lag)

print(f'Scraping terminé ! Durée totale : {round((time.time()-begin)/60, 2)}')
df.to_csv('nba_player_data.csv', index=False)

___

### Nettoyage et préparation des données

Dans cette partie, nous allons retirer certaines données inutiles et renommer certaines colonnes pour obtenir un set de données prêt à être utilisé !

In [2]:
data = pd.read_csv('nba_player_data.csv')
data.drop(columns=['RANK', 'EFF'], inplace=True)
data['season_start_year'] = data['Year'].str[:4].astype(int)
data['Season_type'].replace('Regular%20Season','RS', inplace=True)

C:\Users\rudyv\AppData\Local\Temp\ipykernel_20172\1923326894.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Season_type'].replace('Regular%20Season','RS', inplace=True)


In [3]:
data = pd.read_csv('nba_player_data.csv')
data.drop(columns=['RANK', 'EFF'], inplace=True)
data['season_start_year'] = data['Year'].str[:4].astype(int)
data['Season_type'].replace('Regular%20Season','RS', inplace=True)

C:\Users\rudyv\AppData\Local\Temp\ipykernel_20172\1923326894.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Season_type'].replace('Regular%20Season','RS', inplace=True)


On filtre nos données pour créer deux nouveaux DataFrames `rs_df`et `playoffs_df` pour les saisons régulières et les playoffs respectivement.

In [4]:
rs_df = data[data['Season_type']=='RS']
playoffs_df = data[data['Season_type']=='Playoffs']

In [5]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'AST_TOV', 'STL_TOV', 'season_start_year'],
      dtype='object')

In [6]:
total_columns = ['MIN', 'PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF']

### Comment est la distribution des minutes jouées?

In [7]:
fig = px.histogram(x=rs_df['MIN'], histnorm='percent', nbins=50, title='Distribution des minutes jouées en saison régulière')
fig.update_layout(xaxis_title='Minutes jouées', yaxis_title='Pourcentage de joueurs')
fig.show()

On remarque par exemple que sur toute une saison régulière, 13% des joueurs ont joué moins de 100 minutes au total.

In [8]:
def hist_data(df=rs_df, min_MIN=0, min_GP=0):
    return df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'MIN']/df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'GP']

La fonction `hist_data` qui prend en paramètres un DataFrame `df`, et deux valeurs minimales : `min_MIN` pour le nombre minimum de minutes jouées et `min_GP` pour le nombre minimum de matchs joués.
La fonction retourne les minutes jouées par match pour les joueurs ayant joué au moins `min_GP` matchs et ayant une moyenne de minutes jouées par match d'au moins `min_MIN`.

In [9]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=hist_data(rs_df,50,5), histnorm='percent', name='RS',
                           xbins={'start':0, 'end':48, 'size':1}))
fig.add_trace(go.Histogram(x=hist_data(playoffs_df,5,1), histnorm='percent', name='Playoffs',
                            xbins={'start':0, 'end':48, 'size':1}))
fig.update_layout(title='Distribution des minutes jouées par match', xaxis_title='Minutes par match',
                    yaxis_title='Pourcentage des joueurs', barmode='overlay')
fig.update_traces(opacity=0.50)
fig.show()

### Comment est la distribution des points marqués par matchs ?

In [10]:
def hist_data_points(df=rs_df, min_MIN=0, min_GP=0):
    return df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'PTS']/df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'GP']

In [11]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=hist_data_points(rs_df,50,5), histnorm='percent', name='RS',
                           xbins={'start':0, 'end':35, 'size':1}))
fig.add_trace(go.Histogram(x=hist_data_points(playoffs_df,5,1), histnorm='percent', name='Playoffs',
                            xbins={'start':0, 'end':35, 'size':1}))
fig.update_layout(title='Distribution des points marqués par match', xaxis_title='Points par match',
                    yaxis_title='Pourcentage des joueurs', barmode='overlay')
fig.update_traces(opacity=0.50)
fig.show()

Les deux distributions sont très similaires. On peut remarquer que pour les playoffs, les stars augmentent un peu leur moyenne de points marqués par rapport à la saison régulière (ils brillent encore plus). On remarque également que pour les playoffs, l'écart dans la distribution est encore plus extrême avec encore plus de joueurs marquants peu de paniers.

___

### Comment le jeu a-t-il évolué en 10 années ?

Tout d'abord, on créer un DataFrame agrègeant les données de la saison régulière par année de début de saison, puis on va estimer le nombre de possessions pour chaque année, à l'aide de la formule suivante:
$$
\text{Possessions estimées} = \text{FGA} - \text{OREB} + 0.40 \times \text{FTA} + \text{TOV}
$$

Elle prend en compte plusieurs événements qui affectent le changement de possession :

- $\text{FGA}$ : Tentatives de tirs au panier
- $\text{OREB}$ : Rebonds offensifs
- $\text{FTA}$ : Lancers francs tentés
- $\text{TOV}$ : Balles perdues

On approxime à 40% le pourcentage de rebonds offensifs qui mènent à une deuxième tentative de tir.
Cette estimation sera utile pour évaluer l'efficacité d'une équipe à la fois en attaque et en défense.

Ensuite, on calcule différents pourcentages de réussite pour les tirs, ainsi que d'autres ratios comme le pourcentage de passes décisives par tir réussi. Les statistiques sont ensuite ajustées pour 48 minutes de jeu et présentées dans un graphique interactif.

In [12]:
evolution_df = rs_df.groupby('season_start_year')[total_columns].sum().reset_index()
evolution_df['POSS_estimee'] = evolution_df['FGA'] -evolution_df['OREB'] + 0.40*evolution_df['FTA'] + evolution_df['TOV']
evolution_df = evolution_df[list(evolution_df.columns[0:2]) + ['POSS_estimee'] + list(evolution_df.columns[2:-1])]

evolution_df['FG%'] = evolution_df['FGM']/evolution_df['FGA']
evolution_df['3PT%'] = evolution_df['FG3M']/evolution_df['FG3A']
evolution_df['FT%'] = evolution_df['FTM']/evolution_df['FTA']
evolution_df['AST%'] = evolution_df['AST']/evolution_df['FGM'] # représente le pourcentage de paniers marqués par un joueur qui sont issus d'une passe décisive
evolution_df['FG3A%'] = evolution_df['FG3A']/evolution_df['FGA'] # représente le pourcentage de tirs tentés qui sont des tirs à 3 points
evolution_df['PTS/FGA'] = evolution_df['PTS']/evolution_df['FGA']
evolution_df['3PT/FGM'] = evolution_df['FG3M']/evolution_df['FGM'] # représente le pourcentage de paniers marqués qui sont des tirs à 3 points
evolution_df['FT/FGA'] = evolution_df['FTA']/evolution_df['FGA']
# TRU% = 0.5 * PTS / (FGA + 0.475 * FTA)
evolution_df['TRU%'] = 0.5 * evolution_df['PTS'] / (evolution_df['FGA'] + 0.475 * evolution_df['FTA'])
evolution_df['AST/TO'] = evolution_df['AST'] / evolution_df['TOV'] # représente le ratio entre les passes décisives et les pertes de balle

In [13]:
evolution_par48_df = evolution_df.copy()
for col in evolution_par48_df.columns[2:18] :
    evolution_par48_df[col] = (evolution_par48_df[col]/evolution_par48_df['MIN'])*48*5

evolution_par48_df.drop(columns='MIN', inplace=True)

fig = go.Figure()
for col in evolution_par48_df.columns[1:]:
    fig.add_trace(go.Scatter(x=evolution_par48_df['season_start_year'],
                             y=evolution_par48_df[col], mode='lines+markers', name=col,
                             visible='legendonly'))  # Définir la visibilité sur 'legendonly'
fig.update_layout(title='Evolution des statistiques pour 48 minutes en saison régulière',
                  xaxis_title='Année de début de saison', yaxis_title='Statistique par 48 minutes', height=600)
fig.show()

- La courbe `FG3A%` qui représente le pourcentage de tirs à 3 points présente une forte croissance, ce qui montre que le jeu a bien changé en 10 ans, passant de tirs à une distance moyenne à des tirs longue-distance rapportant plus de points.
- La courbe `TRU%` qui représente l'efficacité des tirs a elle aussi augmenté
- La courbe `3PT/FGM` montre qu'entre 2013 et 2023, le pourcentage de tirs à 3 points parmi les paniers marqués est passé de 20% à 30%, soit une augmentation de 50%, ce qui peut à nouveau laisser penser que les tactiques de jeu concernant les 3 points ont évoluées.
- La courbe `PTS` représentant le nombre de points par match a elle aussi augmenté : les joueurs marquent globalement plus de points par match qu'il y a 10 ans.